In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src import edhrec_requests as edh
from src import funciones

In [2]:
#Primera parte. Una vez tenemos el Data Frame de las Cartas, vamos a mirarlo y luego a limpiarlo
#Lo cargamos desde el fichero
df = pd.read_csv("scryfall_cards.csv")
#Lo miramos para ver que está bonito
df.info()

C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\1723738919.py:3: DtypeWarning: Columns (74,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("scryfall_cards.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105879 entries, 0 to 105878
Data columns (total 87 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   object               105879 non-null  object 
 1   id                   105879 non-null  object 
 2   oracle_id            105805 non-null  object 
 3   multiverse_ids       105879 non-null  object 
 4   mtgo_id              54222 non-null   float64
 5   arena_id             16139 non-null   float64
 6   tcgplayer_id         90738 non-null   float64
 7   name                 105879 non-null  object 
 8   lang                 105879 non-null  object 
 9   released_at          105879 non-null  object 
 10  uri                  105879 non-null  object 
 11  scryfall_uri         105879 non-null  object 
 12  layout               105879 non-null  object 
 13  highres_image        105879 non-null  bool   
 14  image_status         105879 non-null  object 
 15  image_uris       

In [ ]:
#Ahora vamos a quedarnos sólo con las que son legales en Commander. 
# La columna de "legalities" contiene un diccionario en forma de string en la celda.
#Lo que me interesa es que sea "'commander': 'legal'". Con las comillas simples.
df_commander = df[df['legalities'].str.contains("'commander': 'legal'", case=False, na=False)]

#Seleccionamos las columnas que me apetecen
df_clean = df_commander[[
    'name', "type_line", 'set', 'set_name', 'released_at', 'rarity', 'reprint', 'edhrec_rank', 'legalities', "cmc"    
]]

#Creamos una columna nueva con precios, que también es un diccionario en string
df_clean['eur'] = df["prices"].apply(lambda x: funciones.extraer_precio(x, "eur"))
# Convertir los precios a float, si no son None
df_clean['eur'] = pd.to_numeric(df_clean['eur'], errors='coerce')

df_clean.info()
df_clean.head(10)


<class 'pandas.core.frame.DataFrame'>
Index: 96387 entries, 0 to 105878
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         96387 non-null  object 
 1   type_line    96319 non-null  object 
 2   set          96387 non-null  object 
 3   set_name     96387 non-null  object 
 4   released_at  96387 non-null  object 
 5   rarity       96387 non-null  object 
 6   reprint      96387 non-null  bool   
 7   edhrec_rank  91886 non-null  float64
 8   legalities   96387 non-null  object 
 9   cmc          96319 non-null  float64
 10  eur          66546 non-null  float64
dtypes: bool(1), float64(3), object(7)
memory usage: 8.2+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\3182825813.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['eur'] = df["prices"].apply(lambda x: funciones.extraer_precio(x, "eur"))
C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\3182825813.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['eur'] = pd.to_numeric(df_clean['eur'], errors='coerce')


,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur
0,Forest,Basic Land — Forest,blb,Bloomburrow,2024-08-02,common,True,NaN,"{'standard': 'legal', 'future': 'legal', 'hist...",0.0,NaN
1,Fury Sliver,Creature — Sliver,tsp,Time Spiral,2006-10-06,uncommon,False,8979.0,"{'standard': 'not_legal', 'future': 'not_legal...",6.0,0.16
2,Kor Outfitter,Creature — Kor Soldier,zen,Zendikar,2009-10-02,common,False,18479.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,0.23
4,Siren Lookout,Creature — Siren Pirate,xln,Ixalan,2017-09-29,common,False,16750.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,0.11
5,Web,Enchantment — Aura,3ed,Revised Edition,1994-04-11,rare,True,24580.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,0.99
6,Wastewood Verge,Land,pdft,Aetherdrift Promos,2025-02-14,rare,True,6578.0,"{'standard': 'legal', 'future': 'legal', 'hist...",0.0,NaN
7,Surge of Brilliance,Instant,who,Doctor Who,2023-10-13,uncommon,False,6589.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,0.24
8,Obyra's Attendants // Desperate Parry,Creature — Faerie Wizard // Instant — Adventure,woe,Wilds of Eldraine,2023-09-08,common,False,15863.0,"{'standard': 'legal', 'future': 'legal', 'hist...",5.0,0.06
9,Venerable Knight,Creature — Human Knight,eld,Throne of Eldraine,2019-10-04,uncommon,False,16986.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,0.14
10,Wildcall,Sorcery,ugin,Ugin's Fate,2015-01-17,rare,True,17048.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,1.57


In [4]:
#Ahora vamos a ir eliminando los tipos de carta o las cartas que no son relevantes para el análisis
#Aquí vamos a quitar las cartas de tipo "Basic Land", "Atracciones", "Tokens" y demás.
mask = df_clean[df_clean["type_line"].str.contains("Basic Land", na=False)]
maskSnow = df_clean[df_clean["type_line"].str.contains("Basic Snow Land", na=False)]
df_clean.drop(maskSnow.index, inplace=True)
df_clean.drop(mask.index, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91960 entries, 1 to 105878
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         91960 non-null  object 
 1   type_line    91892 non-null  object 
 2   set          91960 non-null  object 
 3   set_name     91960 non-null  object 
 4   released_at  91960 non-null  object 
 5   rarity       91960 non-null  object 
 6   reprint      91960 non-null  bool   
 7   edhrec_rank  91886 non-null  float64
 8   legalities   91960 non-null  object 
 9   cmc          91892 non-null  float64
 10  eur          63866 non-null  float64
dtypes: bool(1), float64(3), object(7)
memory usage: 7.8+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\31274786.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(maskSnow.index, inplace=True)
C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\31274786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(mask.index, inplace=True)


In [5]:
#Ahora vamos a quitar las Atracciones
mask = df_clean[df_clean["type_line"].str.contains("Attraction", na=False)]
df_clean.drop(mask.index, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91873 entries, 1 to 105878
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         91873 non-null  object 
 1   type_line    91805 non-null  object 
 2   set          91873 non-null  object 
 3   set_name     91873 non-null  object 
 4   released_at  91873 non-null  object 
 5   rarity       91873 non-null  object 
 6   reprint      91873 non-null  bool   
 7   edhrec_rank  91799 non-null  float64
 8   legalities   91873 non-null  object 
 9   cmc          91805 non-null  float64
 10  eur          63865 non-null  float64
dtypes: bool(1), float64(3), object(7)
memory usage: 7.8+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\4236453910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(mask.index, inplace=True)


In [6]:
#Ahora las promo de doble cara, que da la casualidad de que son las que tienen el "type_line" a False
#Igualmuente, compruebo aquí también que es así.
df_clean[df_clean["type_line"].isna()].head()

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur
668,"Jinnie Fay, Jetmir's Second // Jinnie Fay, Jet...",NaN,sld,Secret Lair Drop,2024-01-22,rare,True,3156.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,NaN
1920,"Ghalta, Primal Hunger // Ghalta, Primal Hunger",NaN,sld,Secret Lair Drop,2022-12-02,rare,True,515.0,"{'standard': 'legal', 'future': 'legal', 'hist...",NaN,NaN
1926,Tuvasa the Sunlit // Tuvasa the Sunlit,NaN,sld,Secret Lair Drop,2024-12-09,mythic,True,6811.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,NaN
2505,Reckoner Bankbuster // Reckoner Bankbuster,NaN,sld,Secret Lair Drop,2025-02-10,rare,True,3086.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,NaN
3604,Stitch in Time // Stitch in Time,NaN,sld,Secret Lair Drop,2022-04-22,rare,True,5049.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,NaN


In [7]:
#Y aquí las elimino, me quito los nan directamente
df_clean.dropna(subset=['type_line'], how='all', inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91805 entries, 1 to 105878
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         91805 non-null  object 
 1   type_line    91805 non-null  object 
 2   set          91805 non-null  object 
 3   set_name     91805 non-null  object 
 4   released_at  91805 non-null  object 
 5   rarity       91805 non-null  object 
 6   reprint      91805 non-null  bool   
 7   edhrec_rank  91736 non-null  float64
 8   legalities   91805 non-null  object 
 9   cmc          91805 non-null  float64
 10  eur          63856 non-null  float64
dtypes: bool(1), float64(3), object(7)
memory usage: 7.8+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\2840991014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.dropna(subset=['type_line'], how='all', inplace=True)


In [8]:
#Ahora vamos a mirar los NaN que hay en edhrank
df_clean[df_clean["edhrec_rank"].isna()]
#Explico por aquí como pueda, pero las cartas que salen aquí no tienen relevancia
#ya que son cartas que sólo existen en la partida gracias a otras cartas. Algunas, por ejemplo
#son el dorso de otras o el fruto de juntar otras o directamente son una mecánica que se introduce sin necesidad
#de estar en el mazo

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur
620,Carnival Elephant Meteor,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.14
905,"Mishra, Lost to Phyrexia",Legendary Artifact Creature — Phyrexian Artificer,bro,The Brothers' War,2022-11-18,mythic,False,NaN,"{'standard': 'legal', 'future': 'legal', 'hist...",7.0,NaN
2042,Unhinged Beast Hunt,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.11
5467,"Brisela, Voice of Nightmares",Legendary Creature — Eldrazi Angel,sld,Secret Lair Drop,2023-08-10,mythic,True,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",11.0,NaN
8470,Squishy Sphinx Ninja,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.12
...,...,...,...,...,...,...,...,...,...,...,...
97520,"Brisela, Voice of Nightmares",Legendary Creature — Eldrazi Angel,v17,From the Vault: Transform,2017-11-24,mythic,True,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",11.0,NaN
97947,Spooky Clown Mox,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.12
100121,Urza's Dark Cannonball,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.18
103448,Wild Ogre Bupkis,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,0.11


In [ ]:
#En las siguientes comprobaciones de Nan de edhrank me doy cuenta de que ninguna de esas entran dentro
#de la construcción de mazo, asi que a pesar de ser legales me las voy a fumar
df_clean.dropna(subset=["edhrec_rank"], how="all", inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91736 entries, 1 to 105878
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         91736 non-null  object 
 1   type_line    91736 non-null  object 
 2   set          91736 non-null  object 
 3   set_name     91736 non-null  object 
 4   released_at  91736 non-null  object 
 5   rarity       91736 non-null  object 
 6   reprint      91736 non-null  bool   
 7   edhrec_rank  91736 non-null  float64
 8   legalities   91736 non-null  object 
 9   cmc          91736 non-null  float64
 10  eur          63806 non-null  float64
dtypes: bool(1), float64(3), object(7)
memory usage: 7.8+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\2200759003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.dropna(subset=['edhrec_rank'], how='all', inplace=True)


In [10]:
#Ahora voy a comprobar si hay tokens en la tabla
df_clean[df_clean["type_line"].str.contains("Token")]
#No queda ninguno asi que en principio tenemos el df limpio. Igualmente voy a mirar más cosas como los precios nulos

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur


In [11]:
#Ahora voy a mirar los duplicados y tal
df_clean[df_clean["name"].duplicated()].sort_values("name")
#Hay varias repetidas que son redundantes. 
#Creo que me voy a crear otro df para los últimos reprints y otro con las únicas para las primeras impresiones

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur
105407,"""Lifetime"" Pass Holder",Creature — Zombie Guest,unf,Unfinity,2022-10-07,rare,False,10531.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,0.41
64866,A Good Day to Pie,Instant,unf,Unfinity,2022-10-07,common,False,24258.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,0.09
32991,A Tale for the Ages,Enchantment,woe,Wilds of Eldraine,2023-09-08,rare,False,11562.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,NaN
43302,A Tale for the Ages,Enchantment,pwoe,Wilds of Eldraine Promos,2023-09-08,rare,True,11562.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,NaN
83673,A Tale for the Ages,Enchantment,woe,Wilds of Eldraine,2023-09-08,rare,False,11562.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,0.14
...,...,...,...,...,...,...,...,...,...,...,...
72094,"Éowyn, Fearless Knight",Legendary Creature — Human Knight,ltr,The Lord of the Rings: Tales of Middle-earth,2023-06-23,rare,False,2540.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,0.35
101066,"Éowyn, Lady of Rohan",Legendary Creature — Human Noble,ltr,The Lord of the Rings: Tales of Middle-earth,2023-11-03,uncommon,True,6511.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,NaN
59614,"Éowyn, Shieldmaiden",Legendary Creature — Human Knight,ltc,Tales of Middle-earth Commander,2023-11-03,mythic,True,3484.0,"{'standard': 'not_legal', 'future': 'not_legal...",5.0,NaN
9070,"Éowyn, Shieldmaiden",Legendary Creature — Human Knight,ltc,Tales of Middle-earth Commander,2023-06-23,mythic,False,3484.0,"{'standard': 'not_legal', 'future': 'not_legal...",5.0,1.25


In [12]:
#Vamos a limpiar los duplicados

#convierto la columna de released_at 
df_clean["released_at"] = pd.to_datetime(df_clean["released_at"])

#Ordenamos el DataFrame por "released_at" de forma ascendente para tener los viejos primero
df_clean = df_clean.sort_values("released_at")

#elimino duplicados mirando el nombre y quedándome con el primero que aparezca por antigüedad
df_final = df_clean.drop_duplicates(subset=["name"], keep="first")

#y ahora compruebo que df_final está bien
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58574 to 104496
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         28369 non-null  object        
 1   type_line    28369 non-null  object        
 2   set          28369 non-null  object        
 3   set_name     28369 non-null  object        
 4   released_at  28369 non-null  datetime64[ns]
 5   rarity       28369 non-null  object        
 6   reprint      28369 non-null  bool          
 7   edhrec_rank  28369 non-null  float64       
 8   legalities   28369 non-null  object        
 9   cmc          28369 non-null  float64       
 10  eur          25197 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(3), object(6)
memory usage: 2.4+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_12580\3402230447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["released_at"] = pd.to_datetime(df_clean["released_at"])


In [13]:
#Y con esto termino la limpieza de las cartas y un último vistazo
df_final

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur
58574,Jade Statue,Artifact,lea,Limited Edition Alpha,1993-08-05,uncommon,False,26957.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,232.12
45838,Plateau,Land — Mountain Plains,lea,Limited Edition Alpha,1993-08-05,rare,False,478.0,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,3273.87
55829,Disintegrate,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,14350.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,16.08
30484,White Ward,Enchantment — Aura,lea,Limited Edition Alpha,1993-08-05,uncommon,False,23321.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,35.67
1986,Sinkhole,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,16502.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,83.90
...,...,...,...,...,...,...,...,...,...,...,...
31925,Autarch Mammoth,Creature — Elephant Mount,dft,Aetherdrift,2025-02-14,uncommon,False,20864.0,"{'standard': 'legal', 'future': 'legal', 'hist...",6.0,0.06
31947,Wild Roads,Land,dft,Aetherdrift,2025-02-14,uncommon,False,16833.0,"{'standard': 'legal', 'future': 'legal', 'hist...",0.0,0.09
19677,Greenbelt Guardian,Creature — Elf Ranger,dft,Aetherdrift,2025-02-14,uncommon,False,19825.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,0.09
62217,Camera Launcher,Artifact Creature — Construct,dft,Aetherdrift,2025-02-14,common,False,21734.0,"{'standard': 'legal', 'future': 'legal', 'hist...",3.0,0.06


In [14]:
#Ahora vamos a añadir una columna de años de publicación
df_final = edh.gestiona_el_Release(df_final)
df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58574 to 104496
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          28369 non-null  object        
 1   type_line     28369 non-null  object        
 2   set           28369 non-null  object        
 3   set_name      28369 non-null  object        
 4   released_at   28369 non-null  datetime64[ns]
 5   rarity        28369 non-null  object        
 6   reprint       28369 non-null  bool          
 7   edhrec_rank   28369 non-null  float64       
 8   legalities    28369 non-null  object        
 9   cmc           28369 non-null  float64       
 10  eur           25197 non-null  float64       
 11  release_year  28369 non-null  int32         
dtypes: bool(1), datetime64[ns](1), float64(3), int32(1), object(6)
memory usage: 2.5+ MB


e:\AreaTrabajo\REPO_PRUEBA\Itsazain_Bilbao_EDA\src\edhrec_requests.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["released_at"] = pd.to_datetime(dataFrame["released_at"], errors="coerce")
e:\AreaTrabajo\REPO_PRUEBA\Itsazain_Bilbao_EDA\src\edhrec_requests.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["release_year"] = dataFrame["released_at"].dt.year


,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,eur,release_year
58574,Jade Statue,Artifact,lea,Limited Edition Alpha,1993-08-05,uncommon,False,26957.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,232.12,1993
45838,Plateau,Land — Mountain Plains,lea,Limited Edition Alpha,1993-08-05,rare,False,478.0,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,3273.87,1993
55829,Disintegrate,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,14350.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,16.08,1993
30484,White Ward,Enchantment — Aura,lea,Limited Edition Alpha,1993-08-05,uncommon,False,23321.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,35.67,1993
1986,Sinkhole,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,16502.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,83.90,1993


In [15]:
#Y ahora lo voy a pasar a un .csv para no tener que hacer eta puñetera limpieza todo el rato cada vez que reinicio el pc
df_final.to_csv("cartas_limpiadas.csv", index=False)

In [ ]:
df_final